## Задание 9.5. Модуль ML-6 (HW-03)
Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты.

КРИТЕРИИ ОЦЕНИВАНИЯ:

* 1 балл	Верно выделены три столбца-признака для обучения, выбранные RFE.
* 1 балл	Верно выделены три столбца-признака для обучения, выбранные SelectKBest.
* 3 балла	Обучена регрессия на первых трёх столбцах, оценено качество модели на тесте.
* 3 балла	Обучена регрессия на вторых трёх столбцах, оценено качество модели на тесте.
* 2 балла	Произведено сравнение выбранных метрик в форме комментария. Дан ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?» (в текстовой ячейке).

In [213]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

### Загрузка данных

In [214]:
data = pd.read_excel('data/data_ford_price.xlsx')

###  Отбор признаков
### Предобработка данных

In [215]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

### Обучение модели

In [216]:

model_full = LinearRegression()
model_full = model_full.fit(X_train, y_train)

y_test_pred = model_full.predict(X_test)

print('Test-full MAE: {:.2f}'.format(mean_absolute_error(y_test, y_test_pred)))

Test-full MAE: 4682.96


### Удаление избыточного признака

In [217]:
x_m = x.drop('lat', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(x_m, y, test_size=0.3, random_state=40)

m_model = LinearRegression()
m_model.fit(X_train, y_train)
y_predicted = m_model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('Мотивация MAE: %.2f' % mae)

Мотивация MAE: 4672.93


### Метод рекурсивного исключения признаков

In [218]:
from sklearn.feature_selection import RFE

In [219]:
y = data['price']
x = data.drop(columns='price')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [220]:
estimator = LinearRegression()
rfe_selector = RFE(estimator, n_features_to_select=3, step=1)
rfe_selector = rfe_selector.fit(X_train, y_train)
 
rfe_cols = rfe_selector.get_feature_names_out()
display(rfe_cols)

array(['year', 'cylinders', 'lat'], dtype=object)

In [221]:
y = data['price']
x = data[rfe_cols]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

rfe_model = LinearRegression()
rfe_model = rfe_model.fit(X_train, y_train)

y_test_pred = rfe_model.predict(X_test)

print('Test MAE: {:.3f}'.format(mean_absolute_error(y_test, y_test_pred)))

Test MAE: 5096.570


###  Метод выбора признаков на основе фильтров

In [222]:
from sklearn.feature_selection import SelectKBest, f_regression

In [223]:
y = data['price']
x = data.drop(columns='price')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [224]:
skb_selector = SelectKBest(f_regression, k=3)
skb_selector.fit(X_train, y_train)
 
skb_cols = skb_selector.get_feature_names_out()
display(skb_cols)

array(['year', 'cylinders', 'odometer'], dtype=object)

In [226]:
y = data['price']
x = data[skb_cols]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X_train, y_train)
y_test_pred = lin_reg.predict(X_test)

print('Test MAE: {:.2f}'.format(mean_absolute_error(y_test, y_test_pred)))

Test MAE: 4708.95


### Вывод:

#### Произведено сравнение выбранных метрик. Метод RFE отбора признаков показал наилучший результат на тестовой выборке.